In [1]:
import numpy as np
X = np.load('datasets/delivery_review.npy')
y = np.load('datasets/delivery_label.npy')

In [2]:
with open('datasets/chinese_stop_words.txt', encoding='utf-8') as text:
    stopwords = [line.strip() for line in text]

In [3]:
import jieba
X_new = []
for i, review in enumerate(X):
    review = list(jieba.cut(review))
    result = []
    for word in review:
        if word not in stopwords:
            result.append(word)
    X_new.append(" ".join(result))
X_new = np.array(X_new)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BEIZHO~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.874 seconds.
Prefix dict has been built succesfully.


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, shuffle=True)

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = max([len(sentence.split(" ")) for sentence in X_new])
print(maxlen)
words_size = 10000
tokenizer = Tokenizer(num_words=words_size)
tokenizer.fit_on_texts(X_new)

sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, 
                        maxlen=maxlen, 
                        padding='pre')

sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, 
                       maxlen=maxlen, 
                       padding='pre')

Using TensorFlow backend.


135


In [22]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.optimizers import Adam
# 将嵌入层输出词向量的长度指定为32
embedding_dim = 32
model = Sequential()
model.add(Embedding(input_dim=words_size, 
                    input_length=maxlen,
                    output_dim=embedding_dim))
# 一个有256个单元的简单循环神经网络
model.add(SimpleRNN(units=1,
                    return_sequences=False))
model.add(Dense(units=1, 
                activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 135, 32)           320000    
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 1)                 34        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 2         
Total params: 320,036
Trainable params: 320,036
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.layers[1].get_weights()

[array([[-0.19319932],
        [-0.2923085 ],
        [-0.13547462],
        [ 0.03436694],
        [-0.23243415],
        [-0.23891765],
        [-0.02206981],
        [-0.14005175],
        [ 0.01825464],
        [-0.42364272],
        [ 0.08746189],
        [ 0.19246978],
        [-0.09215388],
        [ 0.11909807],
        [ 0.2524215 ],
        [ 0.18055266],
        [ 0.3555318 ],
        [-0.2664271 ],
        [-0.27113497],
        [ 0.10396022],
        [-0.29792687],
        [-0.15469646],
        [ 0.29153132],
        [-0.28954464],
        [-0.27019995],
        [ 0.25052005],
        [-0.36093536],
        [-0.31539667],
        [ 0.02630982],
        [-0.25933188],
        [ 0.4160186 ],
        [-0.4123666 ]], dtype=float32),
 array([[1.]], dtype=float32),
 array([0.], dtype=float32)]

In [7]:
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,
          y_train,
          epochs=5,
          batch_size=32,
          verbose=2,
          validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 5120 samples, validate on 1280 samples
Epoch 1/5
 - 22s - loss: 0.7080 - acc: 0.5281 - val_loss: 0.6895 - val_acc: 0.5047
Epoch 2/5
 - 20s - loss: 0.5239 - acc: 0.7566 - val_loss: 0.3890 - val_acc: 0.8320
Epoch 3/5


KeyboardInterrupt: 